# SQLAlchemy

SQLAlchemy - biblioteka skirta palengvinti darbą su duomenų bazėmis, komunikaciją Python <-> duomenų bazė. Ši biblioteka naudojama kaip ORM (angl. Object Relational Mapper) įrankis, kuris "išverčia" Python aprašytas klases į duomenų bazės lenteles ir automatiškai paverčia funkcijų kvietimus į SQL užklausas. SQLAlchemy biblioteka leidžia programuotojui dirbti neprisirišant prie konkrečios duomenų bazės, nes naudojama viena standartinė sąsaja darbui su daug skirtingų duomenų bazių. 

Kuriama programa <-> SQLAlchemy <-> DBAPI (angl. DataBase API) konkrečios duomenų bazės (MySQL, SQLite, PostgreSQL ir t.t.)

Prieš pradedant dirbti su SQLAlchemy ir duomenų bazėmis reikia sukurti varikliuką (angl. engine), kuris atliks komunikaciją su DBAPI funkcijomis. 

In [ ]:
# sukuriamas variklis darbui su SQLite duomenų baze
from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:", echo=True) # echo=True leis matyti visas SQL komandas

Norint nurodyti duomenų bazės lenteles ir jas atitinkančias Python klases naudojamas deklaratyvi sistema (angl. Declarative system). Šis sistema leidžia susieti lenteles su klasėmis. Tam naudojama Bazinė (angl. Base) klasė, kurios pagalba valdomas klasių ir lentelių sąsaja.

In [ ]:
# sukuriama deklaratyvi bazinė klasė
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

SQLAlchemy biblioteka palaiko visus populiariausius duomenų tipus. Minimalūs reikalavimai kuriant modelius: \_\_tablename\_\_ ir bent vienas stulpelis.

In [ ]:
# deklaratyvios klasės sukūrimas
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

Sukūrus User klasę per deklaratyvią sistemą, aprašėm informaciją apie lentelę, dar vadinamą metaduomenimis (angl. metadata). Sukurtas lentelės objektas yra didesnės kolekcijos MetaData narys. MetaData yra registras visų lentelių objektų ir jis atlieka jų sukūrimą duomenų bazėje.

In [ ]:
User.__table__

In [ ]:
# sukuriama lentelė duomenų bazėje
Base.metadata.create_all(engine)

Dabar Users klasė yra susieta su duomenų bazės lentele.

In [ ]:
# sukuriamas klasės Users objektas
tom_user = User(name="tom", fullname="Tomas Jonaitis", password="tomo_password")

In [ ]:
print(tom_user.name, tom_user.fullname, tom_user.password)

Sukūrus norimas lenteles galima pradėti "šnekėtis" (kurti, redaguoti, trinti, ieškoti įrašų) su duomenų baze. Tai atliekama per sisijas (angl. session).

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# duomenų perdavimas į sesiją
session.add(tom_user)

Kol kas duomenys nėra įrašyti į duomenų bazę, jie yra laukimo (angl. pending) būsenoje. Duomenys į lentelę bus surašyti tik iškvietus duomenų surašymo (angl. commit) komandą. Jei šiuo metu būtų iškviesta duomenų paieškos funkcija, tai laukimo būsenoje esantys duomenys būtų automatiškai surašyti į duomenų bazę. 

Biblioteka užtikrina, jog visos operacijos atliekamos sesijoje operuos tais pačiais duomenimis.

In [ ]:
# atliekama duomenų paieška duomenų bazėje pagal name lauką
our_user = session.query(User).filter_by(name="tom").first()

print(our_user)
print(tom_user is our_user)

In [ ]:
# į sesiją surašomi nauji User objektai add_all([])
session.add_all([
    User(name="wendy", fullname="Wendy Williams", password="foobar"),
    User(name="mary", fullname="Mary Contrary", password="xxg527"),
    User(name="fred", fullname="Fred Flinstone", password="blah")])

In [ ]:
# pakeičiamas vartotojo slaptažodis
tom_user.password = 'f8s7ccs'

Surašytus duomenis į sesiją galima pamatyti:

In [ ]:
print(session.dirty)
print(session.new)

In [ ]:
# inicijuojamas duomenų surašymas į duomenų bazė
session.commit()

In [ ]:
# duomenys surašyti duomenų bazėje, galima pamatyti sugeneruotas laukų reikšmes
print(tom_user.id)

# Duomenų atstatymas (angl. rolling back)